In [79]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


In [80]:
train_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Train.csv')
riders_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Riders.csv')
test_data = pd.read_csv('/Users/rade_dragosavac/PycharmProjects/SendyLogisticsChallenge /Data/Test.csv')


In [81]:
def time_from_midnight_in_seconds(column):
    train_data[column] = pd.to_datetime(train_data[column])
    return (train_data[column] - pd.to_datetime(pd.to_datetime('today').date())).astype('timedelta64[s]')


def delta_time(dataset, higher_time, lower_time):
    return dataset[higher_time] - dataset[lower_time]


def average(list_data):
    return sum(list_data)/len(list_data)+1


In [82]:
# converting time in seconds from midnight
test_data['Placement - Time'] = time_from_midnight_in_seconds('Placement - Time')
test_data['Confirmation - Time'] = time_from_midnight_in_seconds('Confirmation - Time')
test_data['Pickup - Time'] = time_from_midnight_in_seconds('Pickup - Time')
test_data['Arrival at Pickup - Time'] = time_from_midnight_in_seconds('Arrival at Pickup - Time')
test_data['Arrival at Destination - Time'] = time_from_midnight_in_seconds('Arrival at Destination - Time')


In [83]:
# calculating delta_time
delta_confirm_place = delta_time(test_data, 'Confirmation - Time', 'Placement - Time')
delta_pick_arr_confirm = delta_time(test_data, 'Arrival at Pickup - Time', 'Confirmation - Time')
delta_pickup_confirm = delta_time(test_data, 'Pickup - Time', 'Arrival at Pickup - Time')
delta_arrival_pickup = delta_time(test_data, 'Arrival at Destination - Time', 'Pickup - Time')
delta_placement_arrival = delta_time(test_data, 'Arrival at Destination - Time', 'Placement - Time')


In [84]:
test_with_rider_info = test_data.merge(riders_data, on='Rider Id')


In [85]:
list(test_with_rider_info['Placement - Day of Month'] == test_with_rider_info['Confirmation - Day of Month'])
list(test_with_rider_info['Confirmation - Day of Month']) == list(test_with_rider_info['Arrival at Pickup - Day of Month'])
list(test_with_rider_info['Arrival at Pickup - Day of Month']) == list(test_with_rider_info['Pickup - Day of Month'])


True

In [86]:
list(test_with_rider_info['Placement - Weekday (Mo = 1)'] == test_with_rider_info['Confirmation - Weekday (Mo = 1)'])
list(test_with_rider_info['Confirmation - Weekday (Mo = 1)']) == list(test_with_rider_info['Arrival at Pickup - Weekday (Mo = 1)'])
list(test_with_rider_info['Arrival at Pickup - Weekday (Mo = 1)']) == list(test_with_rider_info['Pickup - Weekday (Mo = 1)'])


True

In [88]:
# input missing values
test_with_rider_info['Temperature'].fillna(test_with_rider_info['Temperature'].mean(), inplace=True)


In [89]:
# label encoding of personal / business column
labelencoder_personal_business = LabelEncoder()
test_with_rider_info['Personal or Business'] = labelencoder_personal_business.fit_transform(test_with_rider_info['Personal or Business'])


In [91]:
# one hot encoding of the test_data['Platform Type'] column
test_with_rider_info['Platform Type'] = test_with_rider_info['Platform Type'].astype('category')
test_with_rider_info = pd.concat([test_with_rider_info.drop(columns=['Platform Type']), pd.get_dummies(test_with_rider_info['Platform Type'])], axis=1)


In [92]:
# drop redundant columns
test_with_rider_info.drop(columns=['Confirmation - Day of Month', 'Confirmation - Weekday (Mo = 1)', 
                                   'Arrival at Pickup - Day of Month','Arrival at Pickup - Weekday (Mo = 1)', 
                                   'Pickup - Day of Month', 'Pickup - Weekday (Mo = 1)', 
                                   'Vehicle Type'], inplace=True) 


In [94]:
test_with_rider_info.rename(columns={1: "Platform Type 1", 2: "Platform Type 2", 3: "Platform Type 3", 4: "Platform Type 4"}, inplace=True)


In [ ]:
test_with_rider_info.rename(columns={'Placement - Day of Month': 'Day of Month', 
                                     'Placement - Weekday (Mo = 1)': 'Weekday (Mo = 1)'})


In [102]:
test_with_rider_info['Time from Pickup to Arrival'] = delta_arrival_pickup
